#**Introdution** <br>

##Context
The World Happiness Report is a landmark survey of the state of global happiness . The report continues to gain global recognition as governments, organizations and civil society increasingly use happiness indicators to inform their policy-making decisions. Leading experts across fields – economics, psychology, survey analysis, national statistics, health, public policy and more – describe how measurements of well-being can be used effectively to assess the progress of nations. The reports review the state of happiness in the world today and show how the new science of happiness explains personal and national variations in happiness.<br>


In this part we try to fly over the basic concepts of Causal Inference leveraging Causal Graphs modeling. We start from common-sense assumptions in the form of graphs and we try to fit the dataset to these representation. The main concepts we are going to tackle are:

root nodes and conditional nodes<br>
mechanisms<br>
interventions<br>
colliders and y-shapes<br>
biases in causal graphs<br>
The objective is to get familiar with the principles of causal analysis, the different tools we have to infer values of conditional nodes, how to fit the assumptions in the graph to the dataset and how to apply intervention on nodes. What we are doing here is trying to test different naive and educated guesses and compare them. There is also the possibility of using algorithms to do what is called causal discovery (for example, Fast Causal Inference). There are also combinatorial processes to design the causal graph. These other aspects will be tackled briefly in later posts.

Graphical modeling (gcm) at current time is still an experimental feature in the doWhy library.

# Load the dataset

In [5]:
from functools import partial
def flatten(xss):
    return [x for xs in xss for x in xs]
import pandas as pd
from tabulate import tabulate

In [6]:
from google.colab import files
uploaded = files.upload()

Saving world-happiness-report-2021.csv to world-happiness-report-2021.csv


In [67]:
import io
df = pd.read_csv(io.BytesIO(uploaded['world-happiness-report-2021.csv']))

In [68]:
df.drop(list(df.filter(regex='Explained')), axis=1, inplace=True)
df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,2.798


from matplotlib import pyplot as plt
_df_0['Ladder score'].plot(kind='hist', bins=20, title='Ladder score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Standard error of ladder score'].plot(kind='hist', bins=20, title='Standard error of ladder score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['upperwhisker'].plot(kind='hist', bins=20, title='upperwhisker')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['lowerwhisker'].plot(kind='hist', bins=20, title='lowerwhisker')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_16['lowerwhisker'].plot(kind='line', figsize=(8, 4), title='lowerwhisker')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['Country name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='Ladder score', y='Country name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['Country name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='Standard error of ladder score', y='Country name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['Country name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='upperwhisker', y='Country name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['Country name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='lowerwhisker', y='Country name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

Let's rename the columns so to match our graphs:

Ladder score: Y
Logged (natural) GDP per capita: S
Social support: J
Healthy life expectancy: X
Freedom to make life choices: W

In [10]:
df = df[["Ladder score",
         "Logged GDP per capita",
         "Social support",
         "Healthy life expectancy",
         "Freedom to make life choices"]
].copy()
df.rename(columns={
    "Ladder score": "Y",
    "Logged GDP per capita": "S",
    "Social support": "J",
    "Healthy life expectancy": "X",
    "Freedom to make life choices": "W"
}, inplace=True)
df.head(5)

,Y,S,J,X,W
0,7.842,10.775,0.954,72.0,0.949
1,7.620,10.933,0.954,72.7,0.946
2,7.571,11.117,0.942,74.4,0.919
3,7.554,10.878,0.983,73.0,0.955
4,7.464,10.932,0.942,72.4,0.913


In [11]:
df.describe()

,Y,S,J,X,W
count,149.000000,149.000000,149.000000,149.000000,149.000000
mean,5.532839,9.432208,0.814745,64.992799,0.791597
std,1.073924,1.158601,0.114889,6.762043,0.113332
min,2.523000,6.635000,0.463000,48.478000,0.382000
25%,4.852000,8.541000,0.750000,59.802000,0.718000
50%,5.534000,9.569000,0.832000,66.603000,0.804000
75%,6.255000,10.421000,0.905000,69.600000,0.877000
max,7.842000,11.647000,0.983000,76.953000,0.970000


#Hypothesis 0 <br>
Now that we have a selection of what we wanted as defined in the previous blogpost, we can start running experiments on our assumptions.

Let's start with trying to fit a uneducated guess, the "pauperistic" causal graph (0):

![picture](https://drive.google.com/uc?export=view&id=1XOTv6AQF_miKVw3GJjSWBzg2ZSToJ5uq)

In [15]:
!pip install causal-learn git+https://github.com/py-why/dowhy.git

  Cloning https://github.com/py-why/dowhy.git to /tmp/pip-req-build-ycdodqme
  Running command git clone --filter=blob:none --quiet https://github.com/py-why/dowhy.git /tmp/pip-req-build-ycdodqme
  Resolved https://github.com/py-why/dowhy.git to commit 36fd02c57a152175b66ee09f14d5119602c0e309
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.4/174.4 kB 3.2 MB/s eta 0:00:00
  Created wheel for dowhy: filename=dowhy-0.0.0-py3-none-any.whl size=380924 sha256=833c3b173bc0583f29e3907d3f15fe977993180dc1153d9d87367a29532cd9ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-l9duysaj/wheels/f2/99/c0/0a54f7e46c89232a9e1e21145a4d97804c8f491ed510fbdb8e
Successfully built dowhy


In [16]:
from dowhy import gcm
import networkx as nx

scm0 = gcm.StructuralCausalModel(
    nx.DiGraph([('S', 'Y')])
)
# we draw the mechanism for the root node S by using "a model that uniformly samples from data samples"
scm0.set_causal_mechanism(
    'S', gcm.EmpiricalDistribution())  ## alternative BayesianGaussianMixtureDistribution
scm0.set_causal_mechanism(
    'Y', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

In [17]:
# we defined a statistical mechanism for each node according to a probabilistic distribution
scm0.causal_mechanism("Y")

Let's fit the designed causal graph to the dataset:

In [18]:
gcm.fit(scm0, df[["S", "Y"]])

Fitting causal mechanism of node Y: 100%|██████████| 2/2 [00:00<00:00, 60.64it/s]


Now we are ready for some exploratory activities One of the most important tool to climb the Ladder of Causation is intervention. In this case of one covariate S that defines an effect Y via linear regression what we can expect is to spot more or less association between the two. To step up our understanding we need to do some intervention (in terms of causal analysis do(S)) on the covariate and see what happens. With this basic one covariate effect what we get is a non-causal relation as there is no conditionality involved, S just "transmits" its value to Y according to a mechanism defined statistically on a probability distribution.

Let's try to see what happens with applying an intervention:

In [19]:
def do_intervention_atomic(model, covariate, value):
    "Make an intervention by setting a covariate to a given VALUE"
    return gcm.interventional_samples(
        model,
        {covariate: lambda x: value},
        num_samples_to_draw=149
    )



print("Atomic Intervention: Set value S=n")
# take a random sample of 3 countries
sample_df01 = df[["S", "Y"]].sample(
    n=3, random_state=10101, ignore_index=False).copy()
sample_indices01 = sample_df01.index.copy()

table01 = []
for i in sample_indices01:
    row = [[i]]
    for v in (-1, 0, 5, 9.06, 12):  ## some fixed values to compare
        if v != -1: row.append(
            do_intervention_atomic(
                scm0,
                "S",
                v).iloc[i].to_list()  # <--
        )
        else: row.append(df[["S", "Y"]].iloc[i].to_list())
    table01.append(flatten(row))

print(tabulate(
    table01,
    headers=["i", "S_orig","Y_orig","S=0", "Y_0", "S=5",
             "Y_5", "S=9.6", "Y_9.6", "S=12", "Y_12"]))

Atomic Intervention: Set value S=n
  i    S_orig    Y_orig    S=0        Y_0    S=5      Y_5    S=9.6    Y_9.6    S=12     Y_12
---  --------  --------  -----  ---------  -----  -------  -------  -------  ------  -------
 35     9.859     6.317      0  -1.20206       5  1.08477     9.06  4.93681      12  6.95059
 75    10.189     5.477      0  -1.20725       5  1.08619     9.06  5.64818      12  7.41047
 71     9.94      5.581      0  -0.984098      5  2.8736      9.06  4.68033      12  8.40109


This is an "atomic intervention", the covariate is set to a fixed value.

🔭 it seems that country 75 is much more sensitive to have its income reduce to 1$ than 35 and 71, even they start from a similar original income. This is just a first impression though according to a causal graph that we designed to be limited and flawed according to common sense.

We can also perform "shift interventions" where we apply a function instead of a fixed value.

In [20]:
def do_intervention_shift(model, covariate, func):
    "Make an intervention by setting a covariate by a given FUNCTION"
    return gcm.interventional_samples(
        model,
        {covariate: lambda x: func(log_value=x)},
        num_samples_to_draw=1000
    )

def multiply_by(log_value, multiplier):
    from math import log, e
    return log((e ** log_value) * multiplier)

print("Shifting Intervention [POSSIBLY WRONG]: increase by a percentage")
table02 = []
for i in sample_indices01:
    row = [[i]]
    for v in (-1, 1.1, 1.2, 1.3):
        if v != -1: row.append(
            do_intervention_shift(
                scm0,
                "S",
                partial(multiply_by, multiplier=v)).iloc[i].to_list() ## <---
        )
        else: row.append(df[["S", "Y"]].iloc[i].to_list())
    table02.append(flatten(row))

print(tabulate(
    table02,
    headers=["i", "S_orig","Y_orig", "S_10%", "Y_10%",
             "S_20%", "Y_20%", "S_30%", "Y_30%"]))

Shifting Intervention [POSSIBLY WRONG]: increase by a percentage
  i    S_orig    Y_orig     S_10%    Y_10%     S_20%    Y_20%    S_30%    Y_30%
---  --------  --------  --------  -------  --------  -------  -------  -------
 35     9.859     6.317  11.1483   6.13492   9.75932  6.05774  11.3794  7.24806
 75    10.189     5.477   8.45531  5.62254  11.5243   6.29371  10.0674  6.67048
 71     9.94      5.581   8.45631  5.62347  10.7533   6.72611  11.2854  7.5621


We tried to increase by 10% steps the value for GDP for the three random samples.

🛑 These look like inconsistant outcomes. Maybe another clue that the "pauperistic" casual graph (Hypothesis 0, only S considered as a cause) cannot help in understanding why the effect looks like it looks? Or the model cannot just work with this one covariate setup? As per any other statistical process, it takes many clues to come to a useful result. Maybe we can disproof this hypothesis by showing that the others are better at explaining the effect (Y) that we observe.

#hypothesis 1
Let's start with trying to fit the "naive" causal graph (1), a graph in which all the covariates are effect modifiers (they are all effecting directly):





![picture](https://drive.google.com/uc?export=view&id=1I5B7pmjxYoGAAVl3oFFjeFtnm2vZF7B8)

For consistency of observations we use the same mechanism used for hypothesis 0, a more in depth study of the best mechanisms to use is always advisable though.

In [21]:
columns_order = ["S", "J", "X", "W", "Y"]

scm1 = gcm.StructuralCausalModel(nx.DiGraph([("S", "Y"), ("J", "Y"), ("X", "Y"), ("W", "Y")]))

scm1.set_causal_mechanism(
    'S', gcm.EmpiricalDistribution())
scm1.set_causal_mechanism(
    'J', gcm.EmpiricalDistribution())
scm1.set_causal_mechanism(
    'X', gcm.EmpiricalDistribution())
scm1.set_causal_mechanism(
    'W', gcm.EmpiricalDistribution())
scm1.set_causal_mechanism(
    'Y', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

In [22]:
gcm.fit(scm1, df)

Fitting causal mechanism of node W: 100%|██████████| 5/5 [00:00<00:00, 176.29it/s]


Let's see now what happens if we do the same interventions as for hypothesis 0 by comparing how the other covariates contribute to the score by fixing the values of:

In [23]:
sample_df11 = df.sample(
    n=3, random_state=10101, ignore_index=False).copy()
sample_indices11 = sample_df11.index

original11 = df.iloc[sample_indices11[1]]
# let's try to set the GDP for the the the second sample to 3.0
intervention11_on_s = do_intervention_atomic(
    scm1,
    "S",
    9.06  ## <-- approx 1/3
).iloc[sample_indices11[1]]

table11 = [
    ["original", sample_indices11[1]] + original11[columns_order].to_list(),
    ["S=1/3", sample_indices11[1]] + intervention11_on_s[columns_order].to_list()]

print(tabulate(table11, headers=["action", "i"] + columns_order))

action      i       S      J       X      W        Y
--------  ---  ------  -----  ------  -----  -------
original   75  10.189  0.903  64.703  0.718  5.477
S=1/3      75   9.06   0.848  57.161  0.872  5.99698


As we used the same sample for the previous hypothesis and this one, we can see the difference in the same country sample by doing the same intervention on hypothesis 0 and hypothesis 1.

Atomic Intervention on index 75 in the context of hypothesis 0: Set value to 9.06 (~1/3 of the income)

i    S=9.06  Y_9.06  S=original  Y=original
---  ----  -------   ----------  ----------
75   9.06  4.08419   10.189      5.477

Atomic Intervention on index 75 in the context of hypothesis 1: Set value to 9.06 (~1/3 of the income)

i    S=9.06   Y       S=original  Y=original
---  ------  -------  ----------  ----------
75   9.06    5.48872  10.189      5.477
Trying to keep the rest fixed, we see how the hypothesis 0 is much more "pessimistic" about the association of S to the effect, and this is attributed only to the mere presence of the other effect modifiers. 🛑 Still not very explanatory about causal relations but we can start understanding the basics of graphical causal reasoning. For now the intuition is: the better we can graphically explain the "problem", the clearer will be where to look for meaningful associations. This may be called "graphical causal discovery". Both hypothesis looks quite weak as 0 is flawed by design and 1 results into an increase of Y after the income is reduce to 1/3 of its original value. NOT EVEN CLOSE YET, associations are quite contraddictory and not even a glimpse of an explicative causal relation.

Let's move to a less naive causal graph.

#hypothesis 2<br>
Let's start with trying to fit the "less naive" causal graph (2), a graph with effect modifiers and a mediator:




![picture](https://drive.google.com/uc?export=view&id=1AGytpsZ98EKskkWQK45qrN4YS71RVt5m)


For consistency of observations we use the same mechanism used for hypothesis 0, a more in depth study of the best mechanisms to use is always advisable though.

I bet this is going to look a little better by still not good enough. We will see down the road that we can make a lot of use of patterns in the graph:

"colliders": like for a hypothetical graph A → C ← B --- A and B "collides" in C.
"y-pattern": looks like [A → C ← B, C → D].
So we may want to see and justify the presence of these kinds of patterns in our graph at some point, that is what automated causal discovery algorithms try to do.

In [24]:
# we just need to change one of our edges compared to `scm1``
scm2 = gcm.StructuralCausalModel(nx.DiGraph([("J", "Y"), ("S", "X"), ("X", "Y"), ("W", "Y")]))

scm2.set_causal_mechanism(
    'S', gcm.EmpiricalDistribution())
scm2.set_causal_mechanism(
    'J', gcm.EmpiricalDistribution())
scm2.set_causal_mechanism(
    'X', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))  ## X is not a root node in this one
scm2.set_causal_mechanism(
    'W', gcm.EmpiricalDistribution())
scm2.set_causal_mechanism(
    'Y', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

gcm.fit(scm2, df)

Fitting causal mechanism of node W: 100%|██████████| 5/5 [00:00<00:00, 193.95it/s]


In [25]:
sample_df21 = df.sample(n=3, random_state=10101, ignore_index=False).copy()
sample_indices21 = sample_df21.index

original21 = df.iloc[sample_indices21[1]]
# let's try to set the GDP for the the the second sample to 3.0
intervention21_on_s = do_intervention_atomic(
    scm2,
    "S",
    9.06
).iloc[sample_indices21[1]]

table21 = [
    ["original", sample_indices21[1]] + original21[columns_order].to_list(),
    ["S=1/3", sample_indices21[1]] + intervention21_on_s[columns_order].to_list()]

print(tabulate(table21, headers=["action", "i"] + columns_order))

action      i       S      J        X      W        Y
--------  ---  ------  -----  -------  -----  -------
original   75  10.189  0.903  64.703   0.718  5.477
S=1/3      75   9.06   0.862  65.3866  0.761  5.35804


Still not what we expect from an empirical perspective, we see a relevant movement down of X (life expectancy) when the income is reduced to 1/3, but still its effect on Y is ambiguous, the "Ladder score" (a score of happiness as computed relatively to a fictional baseline country that has all the worst value for every feature) is still improving for sample 75 even with a lower income and a shorter life... How is it possible? We need to trust the data somehow, the problem is probably in the causal graph representation, that is why we are working out to improve it. Maybe we need to try to look for possible colliders or y-shapes?

#hypothesis 3
We are going to try now an hypothesis that contains a collider shape (hypothesis 3, "health and freedom"), and later on we will try to exclude the income from the causal chain by using it as an effect instead of as an indirect cause as in the previous hypothesis.



![picture](https://drive.google.com/uc?export=view&id=1JPJJSneRwLNvCc5cIUZP_asq-ayHQr6P)

There is one collider in X and one mediator in W.

In [26]:
# we have a little more complex graph
scm3 = gcm.StructuralCausalModel(nx.DiGraph([("J", "X"), ("S", "X"), ("S", "W"), ("W", "Y")]))

scm3.set_causal_mechanism(
    'S', gcm.EmpiricalDistribution())
scm3.set_causal_mechanism(
    'J', gcm.EmpiricalDistribution())
scm3.set_causal_mechanism(
    'X', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
scm3.set_causal_mechanism(
    'W', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))  ## W is not a root node in this one
scm3.set_causal_mechanism(
    'Y', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

gcm.fit(scm3, df)

Fitting causal mechanism of node Y: 100%|██████████| 5/5 [00:00<00:00, 216.08it/s]


In [27]:
sample_df31 = df.sample(
    n=3, random_state=10101, ignore_index=False).copy()
sample_indices31 = sample_df31.index

original31 = df.iloc[sample_indices31[1]]

# intervention
intervention31_on_s = do_intervention_atomic(
    scm3,
    "S",
    9.06
).iloc[sample_indices31[1]]

table31 = [
    ["original", sample_indices31[1]] + original31[columns_order].to_list(),
    ["S=1/3", sample_indices31[1]] + intervention31_on_s[columns_order].to_list()]

print(tabulate(table31, headers=["action", "i"] + columns_order))

action      i       S      J        X         W        Y
--------  ---  ------  -----  -------  --------  -------
original   75  10.189  0.903  64.703   0.718     5.477
S=1/3      75   9.06   0.92   62.9649  0.899323  5.45902


Here it starts to sound and look a little closer to what we expected: the "happiness" went down after a lowering in the income that trasmitted both in the "life expecancy" and in the "freedom to choose". Still not good enough but little better. The "social support" reamined unchanged as there is no direct or undirect path from S We should start to think that other factors have a meaningful impact compared to the flawed hypothesis 0? Can we account this minor improvement to the presence of the collider S → X ← J?

Let's try to "control" for another covariate, the "social support" J:

In [28]:
intervention32_on_j = do_intervention_atomic(
    scm3,
    "J",
    0.600
).iloc[sample_indices31[1]]

table32 = [
    ["original", sample_indices31[1]] + original31[columns_order].to_list(),
    ["J=0.6", sample_indices31[1]] + intervention32_on_j[columns_order].to_list()
]

print(tabulate(table32, headers=["action", "i"] + columns_order))

action      i       S      J        X         W        Y
--------  ---  ------  -----  -------  --------  -------
original   75  10.189  0.903  64.703   0.718     5.477
J=0.6      75   8.903  0.6    60.3056  0.723771  5.08311


As we can see "social support" J is a quite relevant variable in this graph, lowering makes some moving down to happen in S, X but the "happiness" seems to not agree and moves up. Again, we need to trust the data so it makes sense to say that this version of the causal graph is not good enough yet again; things look like they are starting to move in the right direction though...

A social support of 0.6 is a statistical anomaly according to the descriptive statistics for J, let's try to use the mean:

In [29]:
intervention33_on_j = do_intervention_atomic(
    scm3,
    "J",
    df["J"].median()
).iloc[sample_indices31[1]]

table33 = [
    ["original", sample_indices31[1]] + original31[columns_order].to_list(),
    ["median J", sample_indices31[1]] + intervention33_on_j[columns_order].to_list()
]

print(tabulate(table33, headers=["action", "i"] + columns_order))

action      i       S      J        X         W        Y
--------  ---  ------  -----  -------  --------  -------
original   75  10.189  0.903  64.703   0.718     5.477
median J   75   9.88   0.832  67.9414  0.892363  6.35865


This starts to look cool! Considering a more realistic value for J the effect seems to go in the direction we expected. Lowering social support to the median value made the income, life expectancy and "happiness" to lower even if of margin that could be irrelevant to the statistical error. Can we say that this causal graph is marginally better at explaining the relations among the variables?

Let's try to control income and freedom to choose to their median values:

In [30]:
s_median = df["S"].median()
w_median = df["W"].median()

intervention34_on_s = do_intervention_atomic(
    scm3,
    "S",
    s_median
).iloc[sample_indices31[1]]
intervention35_on_w = do_intervention_atomic(
    scm3,
    "W",
    w_median
).iloc[sample_indices31[1]]

table34 = [
    ["original", sample_indices31[1]] + original31[columns_order].to_list(),
    [f"S={s_median}", sample_indices31[1]] + intervention34_on_s[columns_order].to_list(),
    [f"W={w_median}", sample_indices31[1]] + intervention35_on_w[columns_order].to_list()
]

print(tabulate(table34, headers=["action", "i"] + columns_order))

action      i       S      J        X         W        Y
--------  ---  ------  -----  -------  --------  -------
original   75  10.189  0.903  64.703   0.718     5.477
S=9.569    75   9.569  0.697  65.0825  0.635975  3.78332
W=0.804    75   7.694  0.877  52.5572  0.804     6.32033


Still a lot to work to do here but we start seeing some sense according to human expectations. These may be good or bad according to if and which biases are present in our interpretation of the relations in the dataset. We are still observing on only one sample, but the model is considering the full dataset while doing its adjustments and we could compare the difference in control mechanism for the entirety of the dataset and maybe tell more.

The presence of a collider and a y-shape can lift us from any doubt, as if confirmed to be fit enough to be in the causal graph it will exclude the presence of an unobserved confouder. Collider shape if spotted using FCI "has a unique independence relationship compared with other causal relationships. In fact, it is one of the “primitives” that constraint-based algorithm, like FCI, looks for. A feature specific to FCI even among constraint-based methods is its ability to discover latent (unobserved) confounders. This is enabled by another primitive, the “Y” structure. Four variables define a “Y” structure when they have the following causal relationships: W1 → X ← W2 and X → Y. Within the “Y” structure, both W1 and W2 are independent of Y conditional on X. This conditional independence helps rule out the possibility of an unmeasured confounder between X and Y. In other words, when FCI finds a “Y” structure in the graph, the causal relationship from X to Y is guaranteed to be unconfounded; otherwise, FCI assumes that possibly unobserved confounders exist."[1]

So if we can demonstrate in the dataset that our y-shape relation exists indeed among S, J, X, Y, we can rule out unobserved confounders [2]. Obviously this takes a great amount of attention, patient and properly acquired and structured datasets from repeatable observations; requirements that we cannot here establish for the sake of this simple example dataset. Teams around the world in any discipline work to make this kind of analysis possible. We will try to apply FCI to the dataset doen the road.

In [32]:
!pip install causalinference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.4 MB/s eta 0:00:00


In [33]:
from causalinference import CausalModel

In [41]:
df = pd.read_csv(io.BytesIO(uploaded['world-happiness-report-2021.csv']))
df.drop(list(df.filter(regex='Explained')), axis=1, inplace=True)
df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,2.798


In [42]:
df = df[["Ladder score",
         "Logged GDP per capita",
         "Social support",
         "Healthy life expectancy",
         "Freedom to make life choices"]
].copy()
df.rename(columns={
    "Ladder score": "Y",
    "Logged GDP per capita": "S",
    "Social support": "J",
    "Healthy life expectancy": "X",
    "Freedom to make life choices": "W"
}, inplace=True)
df.head(5)

,Y,S,J,X,W
0,7.842,10.775,0.954,72.0,0.949
1,7.620,10.933,0.954,72.7,0.946
2,7.571,11.117,0.942,74.4,0.919
3,7.554,10.878,0.983,73.0,0.955
4,7.464,10.932,0.942,72.4,0.913


In [43]:
df.describe()

,Y,S,J,X,W
count,149.000000,149.000000,149.000000,149.000000,149.000000
mean,5.532839,9.432208,0.814745,64.992799,0.791597
std,1.073924,1.158601,0.114889,6.762043,0.113332
min,2.523000,6.635000,0.463000,48.478000,0.382000
25%,4.852000,8.541000,0.750000,59.802000,0.718000
50%,5.534000,9.569000,0.832000,66.603000,0.804000
75%,6.255000,10.421000,0.905000,69.600000,0.877000
max,7.842000,11.647000,0.983000,76.953000,0.970000


#assumptions
These prerequisites must hold:

randomized experiment ("strong" prerequisite)
assignment of treatment must be random:

(Y(0), Y(1)) ⟂ D
unconfounded assumption ("weak" prerequisite)
exclude confounding among covariates (X), there is no unobserved confounder:

(Y(0), Y(1)) ⟂ D|X
Effects of treatment are orthogonal to treatment conditional covariates.

Spotting confounders is not the subject of this post, see Causal Discovery and Causal Graphs about how to avoid confounding.

#initialisation
Let's assign each record to a "treated group" (Y(1)) or to a "control group" (Y(0)):

In [44]:
# randomise treatment in the dataset
import numpy as np
df["D"] = np.random.choice(a=[0,1], size=df["Y"].count(), p=[0.4, 0.6])
print(df["D"].to_numpy())

[0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 1 1 1 1 0 1 0
 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 0
 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 0]


In [45]:
## causalinference uses the convention of calling covariates as `Xn` so we rename for convenience
df.rename(columns={
    "S": "X0",
    "J": "X1",
    "X": "X2",
    "W": "X3"
}, inplace=True)

#intervention 1
Now we simulate the treatment, we increase the "freedom of choice" index (W or X3 depending on which convention we are using) of a given amount only for the treated samples.

First we need to find a way to do that with incurring in errors, let's see how W looks like:

In [46]:
df["X3"].describe()

count    149.000000
mean       0.791597
std        0.113332
min        0.382000
25%        0.718000
50%        0.804000
75%        0.877000
max        0.970000
Name: X3, dtype: float64

There is a standard deviation of 0.113332 so we will go for 1/10 of that just to be sure we don't stir the water too much in the beginning. So our treatment looks like:

Y(1) => X3 = X3 + (std(X3) / 10)
this is the starting scenario:

In [47]:
# let's set aside the initial data and its causal model for comparison
df_start = df.copy()
df_start.head(5)

,Y,X0,X1,X2,X3,D
0,7.842,10.775,0.954,72.0,0.949,0
1,7.620,10.933,0.954,72.7,0.946,0
2,7.571,11.117,0.942,74.4,0.919,0
3,7.554,10.878,0.983,73.0,0.955,0
4,7.464,10.932,0.942,72.4,0.913,0


This is after we apply the treatment ("intervention 1": slight increase of freedom of choice):

In [48]:
std_dev_X3 = df_start["X3"].std()
print(std_dev_X3 / 10)

mask = df_start["D"] == 1

df_intervention1 = df_start.copy()
# apply intervention
df_intervention1.loc[mask, 'X3'] = df_intervention1.loc[mask, "X3"].apply(lambda x: x + (std_dev_X3 / 10))
df_intervention1.head()

0.011333178506605257


,Y,X0,X1,X2,X3,D
0,7.842,10.775,0.954,72.0,0.949,0
1,7.620,10.933,0.954,72.7,0.946,0
2,7.571,11.117,0.942,74.4,0.919,0
3,7.554,10.878,0.983,73.0,0.955,0
4,7.464,10.932,0.942,72.4,0.913,0


We can see that the samples with intervention (D=1) have now the X3 value increased by a minimal ~0.01.

Let's generate the model for **intervention 1**:

In [49]:
# we simplify the model considering only some covariates
causal_interv1 = CausalModel(
    Y=df_intervention1["Y"].to_numpy(),
    D=df_intervention1["D"].to_numpy(),
    X=df_intervention1[["X0", "X1", "X2", "X3"]].to_numpy()
)

Ready to roll!

Let's see some statistics from the observations.

In [50]:
print(causal_interv1.summary_stats)


Summary Statistics

                        Controls (N_c=56)          Treated (N_t=93)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        5.680        1.318        5.444        0.892       -0.236

                        Controls (N_c=56)          Treated (N_t=93)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        9.622        1.187        9.318        1.132       -0.262
             X1        0.823        0.115        0.810        0.115       -0.109
             X2       65.646        7.228       64.599        6.473       -0.153
             X3        0.803        0.124        0.796        0.106       -0.063



A lot to go trough here. N_c is the size of the control group, N_t is the size of the treated group as came out from random sampling. Mean and Standard Deviation are computed for each group for the score and each covariate.

In the upper right Raw-diff is the expected difference between treated and non-treated. This is an absolute difference E[Y(1) - Y(0)], so it is not much explicative and also in this case the difference is lower than the standard deviation.

The interesting part is the normalized differences column Nor-diff for each covariate. This is the Imbens-Rubin (2015) normalized difference in average covariates. It is a relatively complicated equation that the library compute for us, quite useful, and this is just the beginning. Nor-diff is used to spot "covariate imbalance" that happens when treatment and control groups demonstrate insufficient overlap, usually we expect its values to be lower than 0.5 otherwise we may need to apply correction that we are going to explore later like "trimming".

In this case we don't see much of an effect so we will try to "increase the dosage" and push the value to a greater increase.

#intervention 2
We now apply a treatment that is 1/3 of the freedom of choice index:

In [51]:
std_dev_X3 = df_start["X3"].std()
print(std_dev_X3 / 3)

mask = df_start["D"] == 1

df_intervention2 = df_start.copy()
df_intervention2.loc[mask, 'X3'] = df_intervention2.loc[mask, "X3"].apply(lambda x: x + (std_dev_X3 / 3))
df_intervention2.head()

0.03777726168868419


,Y,X0,X1,X2,X3,D
0,7.842,10.775,0.954,72.0,0.949,0
1,7.620,10.933,0.954,72.7,0.946,0
2,7.571,11.117,0.942,74.4,0.919,0
3,7.554,10.878,0.983,73.0,0.955,0
4,7.464,10.932,0.942,72.4,0.913,0


Recompute the causal model:

In [52]:
# we simplify the model considering only some covariates
causal_interv2 = CausalModel(
    Y=df_intervention2["Y"].to_numpy(),
    D=df_intervention2["D"].to_numpy(),
    X=df_intervention2[["X0", "X1", "X2", "X3"]].to_numpy()
)

print(causal_interv2.summary_stats)


Summary Statistics

                        Controls (N_c=56)          Treated (N_t=93)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        5.680        1.318        5.444        0.892       -0.236

                        Controls (N_c=56)          Treated (N_t=93)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        9.622        1.187        9.318        1.132       -0.262
             X1        0.823        0.115        0.810        0.115       -0.109
             X2       65.646        7.228       64.599        6.473       -0.153
             X3        0.803        0.124        0.822        0.106        0.166



As we can see the index Nor-diff for covariate imbalance grew relevantly for X3 quite close to the 0.5 threshold and that exactly what we were expecting, we pushed X3 so much that a a light on our dashboard started blinking, probably intervention 2 is too much of a treatment and we risk to invalidate the critical next step that is **Treatment Effect Estimation**.

#Treatment Effect Estimation
In the previous sections we applied two simulated intervention on a covariate, how do we estimate the expected effects of the interventions? causalinference gives us some tools, we will look into: "Ordinary Least Square (OLS)" and "Matching" to estimate the effect of the intervention on the Ladder Score.

##OLS
Compute OLS for **intervention 1** and **intervention 2**:

In [53]:
causal_interv1.reset()
causal_interv1.est_via_ols()
print(causal_interv1.estimates)

causal_interv2.reset()
causal_interv2.est_via_ols()
print(causal_interv2.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.054      0.091     -0.595      0.552     -0.234      0.125
           ATC     -0.090      0.091     -0.987      0.323     -0.269      0.089
           ATT     -0.033      0.093     -0.354      0.724     -0.216      0.150


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.112      0.094     -1.195      0.232     -0.296      0.072
           ATC     -0.154      0.090     -1.712      0.087     -0.329      0.022
           ATT     -0.087      0.098     -0.887      0.375     -0.280      0.106



/usr/local/lib/python3.10/dist-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]


OLS applies a linear function with covariates adjustments.

Here we are three foundamental indices that are computed by the estimation:

ATE is the Average Treatment Effect: the effect observed in the simulation on the whole dataset
ATC is the Average Effect on Controls: the effect observed in the simulation on the control group
ATT is the Avergae Effect on Treated: the effect observed in the simulation on treated group

This result shows that OLS is essentially imputing the missing potential outcomes of a given group by extrapolating linearly from the observations of the other group. It thus follows that the less covariate overlap there is between the two groups the more hopelessly heroic the extrapolation, especially if the underlying relationship between outcomes and covariates is nonlinear to begin with. (Laurence Wong)

There are plenty of considerations to keep in mind when reading at these numbers. Looking at the column Est. you can have a very rough idea of what is going on between the covariate object of the intervention and the effect. Something we can do is to try a different effect estimation algorithm to see if the extrapolation run with OLS can be adjusted somehow.

#Matching
We can try to compute the estimation on matching records with similar covariate values. We apply the linear extrapolation only on records that have similar characteristics:


In [54]:
causal_interv1.reset()
causal_interv1.est_via_matching(bias_adj=True)
print(causal_interv1.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.022      0.134     -0.165      0.869     -0.285      0.241
           ATC      0.022      0.157      0.139      0.889     -0.286      0.330
           ATT     -0.049      0.145     -0.336      0.737     -0.333      0.235



/usr/local/lib/python3.10/dist-packages/causalinference/estimators/matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef


In [55]:
causal_interv2.reset()
causal_interv2.est_via_matching(bias_adj=True)
print(causal_interv2.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.009      0.131     -0.069      0.945     -0.265      0.247
           ATC     -0.006      0.168     -0.033      0.974     -0.335      0.324
           ATT     -0.011      0.134     -0.083      0.934     -0.273      0.251



We can easily say one thing though, in this estimation we took the whole of the dataset without considering the substantial differences among the records. We may have ended up adding a bias to the estimation by considering all the records at the same time. We are going to apply "stratification" so we can see if the effect estimation changes when applied to more homogeneous subgroups in the dataset.

#propensity score
We are going to create strata in the dataset so to mitigate possible biases. Before we need to introduve the concept of propensity score.

The probability of receiving treatment, also known as the propensity score, plays a very special role in the estimation of treatment effects. (Laurence Wong)

The propensity score is the conditional probability of receiving the treatment given the observed covariates. Estimation is done via a logistic regression. (est_propensity docstring)

Thanks to the unconfounded assumptions (see section above) and the work done by Rosenbaum-Rubin (1983), we can safely assume that this reasoning follows:

# unconfoundedness assumption
(Y(0), Y(1)) ⟂ D
# implies according to Rosembaum-Rubin
(Y(0), Y(1)) ⟂ D | p(X)
# that is
p(X) = P(D=1|X)
This allows to define an algorithm Imbens-Rubin(2015) for variable selection for estimating the propensity score. The propensity score is foundamental to refine the estimation techniques for causal effects.

Let's try to compute the propensity score for the original dataset:

In [56]:
causal_start = CausalModel(
    Y=df_start["Y"].to_numpy(),
    D=df_start["D"].to_numpy(),
    X=df_start[["X0", "X1", "X2", "X3"]].to_numpy()
)

causal_start.est_via_ols()
causal_start.est_via_matching(bias_adj=True)
print(causal_start.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.030      0.091     -0.327      0.744     -0.208      0.148
           ATC     -0.063      0.093     -0.680      0.497     -0.244      0.119
           ATT     -0.010      0.092     -0.105      0.916     -0.189      0.170

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.003      0.132     -0.026      0.979     -0.263      0.256
           ATC      0.029      0.153      0.191      0.848     -0.271      0.329
           ATT     -0.023      0.143     -0.162      0.871     -0.303      0.257



/usr/local/lib/python3.10/dist-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]
/usr/local/lib/python3.10/dist-packages/causalinference/estimators/matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef


In [57]:
print("Appliying only linear logistic regressions")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_start.est_propensity(lin="all")
print(causal_start.propensity)

print("Appliying linear logistic regressions and quadratic for X3 and X3*X1")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_start.est_propensity(lin="all", qua=[(3,3), (1,3)])
print(causal_start.propensity)

Appliying only linear logistic regressions
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      2.337      1.758      1.329      0.184     -1.110      5.783
            X0     -0.585      0.338     -1.728      0.084     -1.249      0.078
            X1      2.498      2.557      0.977      0.329     -2.514      7.509
            X2      0.041      0.051      0.813      0.416     -0.058      0.141
            X3     -1.291      1.801     -0.716      0.474     -4.821      2.240

Appliying linear logistic regressions and quadratic for X3 and X3*X1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
-------------------------------------------------------------------

The search for a good combination of linear (XN) and quadratic (XN*XN) terms for the logistic regression to compute the propensity score is a foundamental way for understanding how the covariates influence each others [1][2].

Negative coefficients in a logistic regression model translate into odds ratios that are less than one (viz., (0,1)). That in turn, means that the predicted probability is decreasing as the covariate increases.

Let's see how this number changes after intervention 2:

In [59]:
print("Appliying only linear logistic regressions")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_interv1.est_propensity(lin="all")
print(causal_interv1.propensity)

Appliying only linear logistic regressions
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      1.981      1.752      1.131      0.258     -1.452      5.414
            X0     -0.576      0.338     -1.704      0.088     -1.239      0.087
            X1      2.078      2.549      0.815      0.415     -2.918      7.074
            X2      0.036      0.051      0.706      0.480     -0.064      0.136
            X3     -0.061      1.777     -0.034      0.973     -3.543      3.422



In [60]:
print("Appliying linear logistic regressions and quadratic for X3 and X3*X1")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_interv2.est_propensity(lin="all", qua=[(3,3), (1,3)])
print(causal_interv2.propensity)

Appliying linear logistic regressions and quadratic for X3 and X3*X1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept     -7.294      8.582     -0.850      0.395    -24.114      9.526
            X0     -0.516      0.347     -1.489      0.137     -1.196      0.164
            X1     12.874     12.791      1.006      0.314    -12.197     37.945
            X2      0.021      0.052      0.402      0.688     -0.081      0.123
            X3     12.150     16.563      0.734      0.463    -20.313     44.614
         X3*X3      1.806     13.212      0.137      0.891    -24.088     27.701
         X1*X3    -15.331     16.172     -0.948      0.343    -47.028     16.365



In [61]:
print("Appliying Imbens-Rubin(2015) algorithm")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_interv2.est_propensity_s()
print(causal_interv2.propensity)

Appliying Imbens-Rubin(2015) algorithm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      1.471      1.600      0.920      0.358     -1.664      4.606
            X0     -0.365      0.171     -2.139      0.032     -0.699     -0.031
            X3      3.067      1.699      1.806      0.071     -0.262      6.396



Note down the differences you notice from before and after intervention 2 and which covariates get influenced by the intervention in X3.

#Trimming
Set a cut-off value to drop values with the propensity score outside a given interval:

In [62]:
# default cut-off is 0.1: only PS between .1 and .9 are considered
# because the min PS is 0.26, we need to set the cut-off high enough
causal_start.cutoff = 0.30

# before trimming
causal_start.reset()
causal_start.est_propensity_s()
causal_start.trim()
print(causal_start.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      2.723      1.447      1.881      0.060     -0.114      5.560
            X0     -0.234      0.151     -1.548      0.122     -0.530      0.062



The dataset looks to be sensitive to dropping tail values.

#Stratification
Another usefull usage of the propensity score is to create buckets of records with inside some interval of PS. With stratification each block is a group in a propensity score interval:

In [63]:
causal_start.reset()
causal_start.est_propensity_s()

causal_start.stratify_s()
print(causal_start.strata)


Stratification Summary

              Propensity Score         Sample Size     Ave. Propensity   Outcome
   Stratum      Min.      Max.  Controls   Treated  Controls   Treated  Raw-diff
--------------------------------------------------------------------------------
         1     0.500     0.763        56        93     0.614     0.630    -0.236



After stratification we are able to compute cleaner estimates for each stratum. This leads us to Blocking.

#Blocking
Aggregating strata estimates of treatment effects gives the blovking estimator, computed by the ATE of every stratum.

With OLS:

In [64]:
causal_start.reset()
causal_start.est_propensity_s()
causal_start.est_via_ols()
print(causal_start.propensity)
print(causal_start.estimates)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      2.723      1.447      1.881      0.060     -0.114      5.560
            X0     -0.234      0.151     -1.548      0.122     -0.530      0.062


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.030      0.091     -0.327      0.744     -0.208      0.148
           ATC     -0.063      0.093     -0.680      0.497     -0.244      0.119
           ATT     -0.010      0.092     -0.105      0.916     -0.189      0.170



With Blocking:

In [65]:
causal_start.reset()
causal_start.est_propensity_s()
causal_start.stratify()  # without trimming
causal_start.est_via_blocking()
print(causal_start.propensity)
print(causal_start.estimates)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      2.723      1.447      1.881      0.060     -0.114      5.560
            X0     -0.234      0.151     -1.548      0.122     -0.530      0.062


Treatment Effect Estimates: Blocking

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.081      0.079      1.026      0.305     -0.074      0.236
           ATC      0.074      0.076      0.979      0.328     -0.074      0.223
           ATT      0.085      0.082      1.039      0.299     -0.076      0.246



We can se the debiasing effect performed by stratification and better estimates.

#Conclusions
With these formally demonstrated tools we should be able to have a clearer understanding about the relations in place with the covariates, permutating over these exercise testing different assumptions can result into new debiased insights. With these new clues we can proceed more confident to a summary of our analysis or rather try to update our causal graph considering the influence that every variable can have on each other, maybe removing edges or establishing colliders and y-shapes in our graph. These methods for relations discovery can be automated via algorithms, that is what we are going to try in the next post.

As an exercise you can try to answer why the causal graph below is, compared to the ones proposed in the previous posts, a better representations of the dataset according to what we have seen until here:

![picture](https://drive.google.com/uc?export=view&id=1ZD5ahWI_EM2l4qBmdU5yL8iVhudTjZP9)

LICENSE¶<br>
MIT License
Copyright (c) 3022 Farheen Zubair
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE

#Reference
[1] Kuss, Oliver. “The z-difference can be used to measure covariate balance in matched propensity score analyses.” Journal of clinical epidemiology vol. 66,11 (2013): 1302-7. doi:10.1016/j.jclinepi.2013.06.001

[2] Austin PC. An Introduction to Propensity Score Methods for Reducing the Effects of Confounding in Observational Studies. Multivariate Behav Res. 2011;46(3):399-424. doi:10.1080/00273171.2011.568786